In [ ]:
# in Windows, run anaconda prompt as admin
# conda install -c conda-forge faiss-cpu (as of 20221008)
# conda install -c services sentence-transformers

In [1]:
import numpy as np
import faiss
import requests
from io import StringIO
import pandas as pd

# Dataset Initialization

In [3]:
res = requests.get('https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/sick2014/SICK_train.txt')

text = res.text
text[:500]

'pair_ID\tsentence_A\tsentence_B\trelatedness_score\tentailment_judgment\n1\tA group of kids is playing in a yard and an old man is standing in the background\tA group of boys in a yard is playing and a man is standing in the background\t4.5\tNEUTRAL\n2\tA group of children is playing in the house and there is no man standing in the background\tA group of kids is playing in a yard and an old man is standing in the background\t3.2\tNEUTRAL\n3\tThe young boys are playing outdoors and the man is smiling nearby\tThe '

We need this in a dataframe, which we build from the `text` string like so:

In [4]:
data = pd.read_csv(StringIO(text), sep='\t')
data.head()

,pair_ID,sentence_A,sentence_B,relatedness_score,entailment_judgment
0,1,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,4.5,NEUTRAL
1,2,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,3.2,NEUTRAL
2,3,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,4.7,ENTAILMENT
3,5,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,3.4,NEUTRAL
4,9,The young boys are playing outdoors and the ma...,A group of kids is playing in a yard and an ol...,3.7,NEUTRAL


We will take all samples from `sentence_A` and build sentence embeddings for each - which we can then store in FAISS.

In [5]:
sentences = data['sentence_A'].tolist()
sentences[:5]

['A group of kids is playing in a yard and an old man is standing in the background',
 'A group of children is playing in the house and there is no man standing in the background',
 'The young boys are playing outdoors and the man is smiling nearby',
 'The kids are playing outdoors near a man with a smile',
 'The young boys are playing outdoors and the man is smiling nearby']

And we will pull in the `sentence_B` column too, giving us ~4.5K *unique* sentences.

In [5]:
sentence_b = data['sentence_B'].tolist()
sentences.extend(sentence_b)
len(set(sentences))

4802

This isn't a particularly large number, so let's pull in a few more similar datasets.

In [8]:
urls = [
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/MSRpar.train.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/MSRpar.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2013/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2014/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2014/images.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2015/images.test.tsv'
]

In [11]:
for url in urls:
    res = requests.get(url)
    # extract to dataframe
    data = pd.read_csv(StringIO(res.text), sep='\t', header=None, error_bad_lines=False)
    # add to columns 1 and 2 to sentences list
    sentences.extend(data[1].tolist())
    sentences.extend(data[2].tolist())

C:\Users\gremw\AppData\Local\Temp\ipykernel_33796\3737715271.py:4: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv(StringIO(res.text), sep='\t', header=None, error_bad_lines=False)
b'Skipping line 191: expected 3 fields, saw 4\nSkipping line 206: expected 3 fields, saw 4\nSkipping line 295: expected 3 fields, saw 4\nSkipping line 695: expected 3 fields, saw 4\nSkipping line 699: expected 3 fields, saw 4\n'
C:\Users\gremw\AppData\Local\Temp\ipykernel_33796\3737715271.py:4: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv(StringIO(res.text), sep='\t', header=None, error_bad_lines=False)
b'Skipping line 104: expected 3 fields, saw 4\nSkipping line 181: expected 3 fields, saw 4\nSkipping line 317: expected 3 fields, saw 4\nSkipping line 412: expected 3 fields, saw 5\nSki

In [13]:
len(set(sentences))

12849

Before converting to our sentence embeddings, we will save to text file as backup.

In [15]:
# remove duplicates and NaN
sentences = [
    sentence.replace('\n', '') for sentence in list(set(sentences)) if type(sentence) is str
    ]

In [16]:
len(sentences)

12848

In [17]:
with open('sentences.txt', 'w') as fp:
    fp.write('\n'.join(sentences))

Now we have 14.5K *unique* sentences, a much better size. We'll go ahead and build the sentence embeddings (this can take some time, feel free to download the embeddings from [here]()). TODO add link

In [19]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('bert-base-nli-mean-tokens')

sentence_embeddings = model.encode(sentences)
sentence_embeddings.shape

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

(12848, 768)

We can save/load from file in the case of needing to reload the notebook for any reason later.

In [25]:
sentence_embeddings.shape[0]

12848

In [27]:
sentence_embeddings[:5]

array([[-9.6647695e-02,  4.6723583e-01,  3.6893499e-01, ...,
         1.1601105e-01, -2.5436433e-02,  4.5156029e-01],
       [ 9.1679931e-01,  5.1914793e-01, -6.4101094e-01, ...,
        -2.9355940e-01,  1.4221726e-01,  4.4613871e-01],
       [-5.8313829e-01,  2.6698965e-01,  2.2379859e-01, ...,
         9.0821609e-02, -5.0137863e-02, -1.4212216e-01],
       [ 3.7224758e-01, -5.7472223e-01,  2.3755560e+00, ...,
        -4.3555823e-01, -5.3475142e-01, -1.0316626e-03],
       [ 1.8525401e-01, -1.3984743e-01,  7.8827852e-01, ...,
         1.4278315e+00, -3.6858153e-02, -8.6588544e-01]], dtype=float32)

In [29]:
with open(f'./sim_sentences/embeddings_X.npy', 'wb') as fp:
    np.save(fp, sentence_embeddings[0:256])

In [30]:
# saving data
split = 256
file_count = 0
for i in range(0, sentence_embeddings.shape[0], split):
    end = i + split
    if end > sentence_embeddings.shape[0] + 1:
        end = sentence_embeddings.shape[0] + 1
    file_count = '0' + str(file_count) if file_count < 0 else str(file_count)
    with open(f'./sim_sentences/embeddings_{file_count}.npy', 'wb') as fp:
        np.save(fp, sentence_embeddings[i:end, :])
    print(f"embeddings_{file_count}.npy | {i} -> {end}")
    file_count = int(file_count) + 1

embeddings_0.npy | 0 -> 256
embeddings_1.npy | 256 -> 512
embeddings_2.npy | 512 -> 768
embeddings_3.npy | 768 -> 1024
embeddings_4.npy | 1024 -> 1280
embeddings_5.npy | 1280 -> 1536
embeddings_6.npy | 1536 -> 1792
embeddings_7.npy | 1792 -> 2048
embeddings_8.npy | 2048 -> 2304
embeddings_9.npy | 2304 -> 2560
embeddings_10.npy | 2560 -> 2816
embeddings_11.npy | 2816 -> 3072
embeddings_12.npy | 3072 -> 3328
embeddings_13.npy | 3328 -> 3584
embeddings_14.npy | 3584 -> 3840
embeddings_15.npy | 3840 -> 4096
embeddings_16.npy | 4096 -> 4352
embeddings_17.npy | 4352 -> 4608
embeddings_18.npy | 4608 -> 4864
embeddings_19.npy | 4864 -> 5120
embeddings_20.npy | 5120 -> 5376
embeddings_21.npy | 5376 -> 5632
embeddings_22.npy | 5632 -> 5888
embeddings_23.npy | 5888 -> 6144
embeddings_24.npy | 6144 -> 6400
embeddings_25.npy | 6400 -> 6656
embeddings_26.npy | 6656 -> 6912
embeddings_27.npy | 6912 -> 7168
embeddings_28.npy | 7168 -> 7424
embeddings_29.npy | 7424 -> 7680
embeddings_30.npy | 7680 -> 7

In [32]:
sentence_embeddings.shape[0]

12848

We setup our FAISS database dimensionality (number of dimensions per vector) based on these vectors.

In [68]:
d = sentence_embeddings.shape[1]
d

768

# Flat L2 Index

We initialize the flat L2 distance index `IndexFlatL2`, all we need is the specify the vector dimensionality - which in this case is `d == 768` (to align with the sentence-BERT model output embeddings of size `768`).

In [69]:
index = faiss.IndexFlatL2(d)

In [70]:
type(index)

faiss.swigfaiss.IndexFlatL2

In [71]:
index

<faiss.swigfaiss.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x000001DA8005DDB0> >

Often, we will use indexes that require us to `train` them on our data before being used (if we are grouping or transforming the data in any way). `IndexFlatL2` however, is a simple operation and only requires that we calculate distances between vectors when we introduce our query vector `xq` during search. So, in this case, no training is required - which we can confirm by checking the `is_trained` attribute.

In [72]:
index.is_trained

True

Okay so once we're happy that our index is prepared, we then add new vectors using the `add` method.

In [73]:
index.add(sentence_embeddings)

In [74]:
index.ntotal

12848

Then search given a query `xq` and number of nearest neigbors to return `k`.

In [75]:
k = 4
xq = model.encode(["Someone sprints with a football"])

In [76]:
%%time
D, I = index.search(xq, k)  # search
print(I)  # k-nearest neigbors of the query vector | nprobe == 1: 6495 26392 61709 49932 | nprobe == 10: 36245  6495 57489  8705

[[5155 1477 7280 2089]]
CPU times: total: 93.8 ms
Wall time: 14.1 ms


Here we're returning indices `7460`, `10940`, `3781`, and `5747`, which returns:

In [77]:
D

array([[54.62374 , 54.853504, 57.356274, 58.870564]], dtype=float32)

In [80]:
[f'{i}: {sentences[i]}' for i in I[0]]

['5155: A group of football players is running in the field',
 '1477: A group of people playing football is running in the field',
 '7280: Two groups of people are playing football',
 '2089: A football player kicks the ball.']

In [82]:
sentences[5155]

'A group of football players is running in the field'

Clearly we have some good matches, everything returned includes people running with a football, or on the context of a football match. Now, if we'd rather extract the numerical vectors from FAISS, we can do that too.

In [83]:
vecs = np.zeros((k, d))
for i, val in enumerate(I[0].tolist()):
    vecs[i, :] = index.reconstruct(val)

In [84]:
vecs.shape

(4, 768)

In [85]:
vecs[0][:100]

array([ 0.01627024,  0.22325903, -0.15037397, -0.30747244, -0.27122411,
       -0.10593186, -0.06460925,  0.04738237, -0.73349077, -0.37657702,
       -0.76762813,  0.16902898,  0.53107667,  0.51176614,  1.14415836,
       -0.08562844, -0.67240077, -0.966371  ,  0.02545465, -0.21559812,
       -1.25656605, -0.829822  , -0.09824988, -0.2185089 ,  0.50610238,
        0.10527924,  0.50396854,  0.65242964, -1.39458692,  0.65847474,
       -0.2152534 , -0.22487433,  0.8181836 ,  0.08464295, -0.76141697,
       -0.28928307, -0.09825767, -0.73046178,  0.07855812, -0.84354568,
       -0.59242064,  0.77471352, -1.20920539, -0.22757988, -1.30733621,
       -0.23081498, -1.31322515,  0.01629087, -0.97285461,  0.19308169,
        0.47424558,  1.18920898, -1.96741283, -0.70061105, -0.2963874 ,
        0.60533714,  0.62407476, -0.70340377, -0.86754155,  0.17673109,
       -0.19170527, -0.0295194 ,  0.22623539, -0.16695416, -0.80402541,
       -0.4591893 ,  0.69675493, -0.24928212, -1.0147866 , -0.92

---

# Adding Partitioning to the Index

FAISS allows us to add an additional step to optimize our search efficiency using a variety of different methods. A popular approach is to partition the index into *[Voronoi cells](https://en.wikipedia.org/wiki/Voronoi_diagram)*. Using this method we would take our query vector `xq`, identify the *cell* it belongs to, and then use our `IndexFlatL2` to search between the query vector `xq` and all indexed vectors belonging to that *cell*. We can also include vectors from other nearby cells too.

We initialize our new partitioned index by first adding our previous `IndexFlatL2` operation as a quantization step (another step in the search process), and feeding this into the new `IndexIVFFlat` operation like so:

In [86]:
nlist = 50
quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFFlat(quantizer, d, nlist)

Here we've added a new parameter `nlist`. We use `nlist` to define how many partitions we'd like our index to have. 

When we built the previous, `IndexFlatL2`-only index, we noted that no training was required as no grouping/transformation was required to build that index. Now that we've added partitioning using `IndexIVFFlat`, this is no longer the case. Let's take a look at the `is_trained` attribute.

In [87]:
index.is_trained

False

So, what we need to do now is `train` our index on our data, which we do *before* adding any data to the index (otherwise the index cannot know how to group each vector).

In [88]:
index.train(sentence_embeddings)
index.is_trained

True

Now our index is trained, we add our data just as we did before.

In [89]:
index.add(sentence_embeddings)
index.ntotal

12848

Let's search again using the same indexed sentence embeddings and the same query `xq`.

In [95]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[5155 1477 7280 2089]]
CPU times: total: 46.9 ms
Wall time: 2.68 ms


We can increase the number of nearby cells to search too with `nprobe`.

In [103]:
index.nprobe = 2

In [104]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[5155 1477 7280 2089]]
CPU times: total: 0 ns
Wall time: 4.47 ms


Increasing the number of `nprobe` will improve the accuracy of our search, but cost time. Our earlier `IndexFlatL2`-only search was *exhaustive* (it compared every single vector) and so it identified the closest matches with a perfect accuracy. The smaller our `nprobe` value, the smaller scope that we search. We received perfect results (that matched our previous `IndexFlatL2`-only results - `7460`, `10940`, `3781`, `5747`), however, if we found that we were not getting closely matching results, we could simply bump `nprobe` up further - improving accuracy, but increasing time-taken too.

It's worth noting that the time taken can change with each run too, if we rerun the above block, we usually get a different time:

In [105]:
%%time
D, I = index.search(xq, k)
print(I)

[[5155 1477 7280 2089]]
CPU times: total: 62.5 ms
Wall time: 3.13 ms


For IVF (and IMI) indexes, before attempting to use the `reconstruct` method, we need to call the `make_direct_map` method - otherwise we will return a `RunetimeError`.

In [106]:
index.reconstruct(7460)

RuntimeError: Error in __int64 __cdecl faiss::DirectMap::get(__int64) const at D:\bld\faiss-split_1613625922506\work\faiss\invlists\DirectMap.cpp:85: direct map not initialized

With `make_direct_map`:

In [107]:
index.make_direct_map()

In [108]:
index.reconstruct(7460)[:100]

array([-2.53096849e-01, -2.53371239e-01,  4.62723851e-01,  2.72474557e-01,
       -1.82537049e-01, -2.44593978e-01,  2.46149510e-01, -5.41305542e-01,
        3.19927722e-01, -3.23612750e-01,  1.27549395e-01,  8.02246392e-01,
       -1.48230180e-01,  9.33590606e-02, -1.03844798e+00,  5.34105062e-01,
       -9.38271657e-02, -3.89643490e-01, -5.15782058e-01,  1.66627899e-01,
        1.29660666e-01, -1.22047566e-01,  3.81940678e-02,  3.05460662e-01,
        8.77040327e-01,  2.05136776e-01,  3.00248154e-02, -3.26555997e-01,
       -8.86763811e-01,  3.64780307e-01,  1.68430969e-01,  8.41970682e-01,
       -1.27686083e+00,  2.03377813e-01,  3.77641737e-01,  9.30500269e-01,
        1.06446075e+00, -2.00449586e-01, -2.17186883e-02,  1.17594987e-01,
       -1.79340579e-02,  5.59337325e-02, -2.81759411e-01,  3.30396473e-01,
       -1.65603533e-01, -7.68142223e-01,  1.29712510e+00,  5.71367264e-01,
        3.91046166e-01, -1.33302402e+00,  3.66828322e-01,  9.78994846e-01,
        2.63698995e-01,  

We've now significantly reduced the search time, what can we do next?

# Quantization

Well, when storing these vectors we're storing the full (eg `Flat`) vector. Now in very big datasets this can quickly become a problem. Typically, we look at big datasets, and when working with large dataset we will find that storing the full vectors consumes too much space.

Fortunately, FAISS comes with the ability to *compress* our vectors using transformations based on *Product Quantization* (PQ). But, what is PQ? Well, we can view it as an additional approximation step similar to our use of **IVF**, which allowed us to *approximate* by reducing the scope of our search. PQ is slightly different however, and approximates the distance (or similarity) calculation instead.

[PQ explanation TODO REMOVE](https://mccormickml.com/2017/10/13/product-quantizer-tutorial-part-1/)

PQ achieves this approximated distance operation by compressing the vectors themselves. This consists of a few steps.

1. We split the every original vector into several subvectors.

2. For each set of subvectors, we perform a clustering operation, creating many centroids for each subvector set.

3. In our vector of subvectors, we replace each subvector with the ID of it's nearest centroid.

In [109]:
m = 8  # number of centroid IDs in final compressed vectors
bits = 8 # number of bits in each centroid

quantizer = faiss.IndexFlatL2(d)  # we keep the same L2 distance flat index
index = faiss.IndexIVFPQ(quantizer, d, nlist, m, bits) 

Again, we'll need to `train` the index.

In [110]:
index.is_trained

False

In [111]:
index.train(sentence_embeddings)

And `add` our vectors.

In [112]:
index.add(sentence_embeddings)

Let's compare it to our previous index *without* PQ, and an `nprobe` value of `10`.

In [113]:
index.nprobe = 10

In [115]:
%%time
D, I = index.search(xq, k)
print(I)

[[ 245  469 2089 1477]]
CPU times: total: 0 ns
Wall time: 1.84 ms


Through adding PQ we've reduced our search time from ~7.5ms to ~5ms, a small difference on a dataset of this size, but when scaled to larger datasets this can make a huge difference.

Now, we should also notice the slightly different results being returned. Beforehand with our exhaustive L2 search we were returning `7460`, `10940`, `3781`, and `5747`. Now, we see a slightly different order to our results - and two different vectors, `5013` and `5370`.

Each of our speed optimization operations, `IVF` and `PQ`, come at the cost of accuracy. Now, if we print out these results we will nonetheless find that each item is still a relevant match:

In [116]:
[f'{i}: {sentences[i]}' for i in I[0]]

['245: A man in a football uniform is running with a football during a game.',
 '469: A group of football players running down the field.',
 '2089: A football player kicks the ball.',
 '1477: A group of people playing football is running in the field']

So although we might not get the *perfect* result, we still get close - and thanks to the approximation, we get a significant speed boost